<a href="https://colab.research.google.com/github/drewbecker02/projects/blob/main/FORSTFinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""Lincoln Tract Project"""
!pip install deap

from deap import tools, creator, base, algorithms
import random
import numpy
import math

IND_SIZE = 87
POP_SIZE = 200
TMIN = 0
TMAX = 6
tract = []
CXPB= 0.0
MUTPB = 0.2
NGEN = 50
final= []
class Stand:
  def __init__(self, ID, area, age, vacre, tcut, adjmatrix):
    """Initializes stand data for the master representation"""
    self.ID = ID
    self.area = area
    self.age = age
    self.vacre = vacre
    self.tcut = tcut
    self.adjmatrix = adjmatrix
  def areaCut(self) -> float:
    """Returns the area cut in the stand at the time in tcut"""
    if self.tcut == 0:
      return 0
    time_cut = self.tcut - 1
    return float(self.area) * float(self.vacre[time_cut])

def feasibleStand(ind, i):
  """Checks a stand for adjacency violations"""
  for n in tract[i].adjmatrix:
    if ind[i] == ind[n-1]:
      return False
  return True

def init_Tract(stand_data, adj_matrix):
  """Initializes the tract that stores the Master Represenatation of the tract"""
  matrix = []
  with open(stand_data) as f:
    lines = f.readlines()
  #assigning stand data to stand classes, populate tract
  for line in lines:
    line = line.rstrip()
    line = line.split(",")
    stand = Stand(line[0], line[1], line[2], [line[3],],\
                  random.randint(TMIN, TMAX), [])
    #assigning volume per acre
    for n in range(4,len(line)):
      vacre = line[n]
      stand.vacre[len(stand.vacre):] = [vacre]
    tract[len(tract):] = [stand]
  #popiulate adjacency matrices
  with open(adj_matrix) as f:
    ajd_lines = f.readlines()
  for line in ajd_lines:
    res = tuple(map(int, line.split(',')))
    matrix[len(matrix):] = [res]
  for stand in tract:
    for pair in matrix:
        if pair[0] == int(stand.ID):
          stand.adjmatrix[len(stand.adjmatrix):] = [pair[1]]

def totalCutPerT(ind, n):
  """Returns total harvest volume of the given time period"""
  if n ==0:
    return 0
  score = 0
  tract_copy = tract.copy()
  for i in range(IND_SIZE):
    stand = tract_copy[i]
    stand.tcut = ind[i]
    if stand.tcut == n:
      score += stand.areaCut()
  return score

def adjViolated(ind):
  """Returns number of adjacency constraints violated"""
  score = 0
  for i in range (IND_SIZE):
    stand = tract[i]
    for n in stand.adjmatrix:
      if stand.tcut > 0 and tract[n-1].tcut == stand.tcut:
        score +=1
  return score

def evaluate(ind):
  """Evaluation function"""

  def nullifyYoungStands(ind):
    """Helper function that changes stands with area cut = 0 to treatment 0"""
    for i in ind:
      if tract[i].areaCut == 0:
        ind[i] = 0
    return ind

  score=0
  ind = nullifyYoungStands(ind)
  for t in range(1, TMAX+1):
    score += totalCutPerT(ind, t)
  #penalties
  if not feasible(ind):
    score = score - 10000 * (adjViolated(ind))
  for t in range (1, TMAX+1):
    if not inBounds(ind, t):
      score = score - 100 * sumOutOfBounds(ind, t)
  if not evenFlow(ind):
    score = score - evenFlowDifference(ind)
  return (score,)
    
def evenFlow(ind):
  """Checks to see if there is even flow over the time periods"""
  total_flow = 0
  for num in range(1, TMAX+1):
    total_flow += totalCutPerT(ind, num)
  avg_flow = total_flow/TMAX
  for num in range(1, TMAX+1):
    if totalCutPerT(ind, num) > 1.2 * avg_flow\
    or totalCutPerT(ind, num) < .8 * avg_flow:
      return False
  return True

def evenFlowDifference(ind):
  """Calculates the difference between the avg flow and the actual flow"""
  sum = 0
  total_flow = 0
  for num in range(1, TMAX+1):
    total_flow += totalCutPerT(ind, num)
  avg_flow = total_flow/TMAX
  for num in range(1, TMAX+1):
    if totalCutPerT(ind, num) > 1.2 * avg_flow\
    or totalCutPerT(ind, num) < .8 * avg_flow:
      sum += abs(totalCutPerT(ind, num) - avg_flow)
  return sum


def sumOutOfBounds(ind, t):
  """Returns the difference between the target volume and the
  harvest volume"""
  return abs(13950 - totalCutPerT(ind, t))
  

def inBounds(ind, t):
  """Checks if the amount cut in t is within our maximum harvest volume"""
  volume = 0
  if totalCutPerT(ind, t) > 13950:
    return False
  return True

def feasible(ind):
  """Checks adjacency constraints"""
  for i in range (IND_SIZE):
    stand = tract[i]
    for id in stand.adjmatrix:
      if tract[id-1].tcut == stand.tcut and \
      float(tract[id-1].vacre[tract[id-1].tcut-1]) > 0.0:
          return False
  return True

def cxGeo(ind1, ind2):
  """New crossover that operates by taking a geographic segment 
  of parent one and a geographic segment of parent 2 to produce two unique children
  EACH CHILD """
  def addCycle(parent, child, i, count, search_list=[], removed=[]):
    """Helper function that recurses through the list of adjacencies and adds stands"""
    if count == 0:
      return child
    else:
      parent_stand = tract[i]
      if child[i] == None:
        child[i] = parent[i]
      for stand in parent_stand.adjmatrix:
        #Create search list to breadth-first search the adjacency tree
        if stand not in search_list and stand not in removed:
          search_list += [stand]
      for stand in search_list:
         removed += [search_list.pop(0)]
      return addCycle(parent, child, stand-1, count-1, search_list, removed)

  c1 = ind1.copy()
  c2 = ind2.copy()
  for i in range(IND_SIZE):
    c1[i] = None
    c2[i] = None
  index = random.randint(0, IND_SIZE-1)
  size = random.randint(0, IND_SIZE)
  #Add adjacency cycle to child 1
  addCycle(ind1, c1, index, size)
  for i in range(IND_SIZE):
    if c1[i]==None:
      c1[i] = ind2[i]
  #add adjacency cycle to child 2
  addCycle(ind2, c2, index, size)
  for i in range(IND_SIZE):
    if c2[i]==None:
      c2[i] = ind1[i]
  #edit and return parents
  for i in range(IND_SIZE):
    ind1[i] = c1[i]
    ind2[i] = c2[i]
  return (ind1, ind2)

def cxSmartGeo(ind1, ind2):
  def addSmartCycle(parent, child, i, count, search_list=[], removed=[]):
    if count == 0:
      return child
    else:
      parent_stand = tract[i]
      if feasibleStand(parent, i) and child[i] == None:
        child[i] = parent[i]
        # print ("p:",parent)
        # print ("c:", child)
      for stand in parent_stand.adjmatrix:
        #Create search list to breadth-first search the adjacency tree
        if stand not in search_list and stand not in removed:
          search_list += [stand]
      for stand in search_list:
        removed += [search_list.pop(0)]
      return addSmartCycle(parent, child, stand-1, count-1, search_list, removed)

  c1 = ind1.copy()
  c2 = ind2.copy()
  for i in range(IND_SIZE):
    c1[i] = None
    c2[i] = None

  index = random.randint(0, IND_SIZE-1)
  for i in range (index, IND_SIZE):
    if i == IND_SIZE-1:
      i = 1
    if feasibleStand(ind1, i):
      index = i
      break
  #add valid stands
  size = random.randint(0, IND_SIZE)
  addSmartCycle(ind1, c1, index, size)
  for i in range(IND_SIZE):
    if c1[i]==None:
      c1[i] = ind2[i]
  for i in range (index, IND_SIZE):
    if i == IND_SIZE-1:
      i = 1
    if feasibleStand(ind2, i):
      index = i
      break
  addSmartCycle(ind2, c2, index, size)
  for i in range(IND_SIZE):
    if c2[i]==None:
      c2[i] = ind1[i]
  for i in range(IND_SIZE):
    ind1[i] = c1[i]
    ind2[i] = c2[i]
  return (ind1, ind2)
  
def createEA():
  global final
  #Making containers
  creator.create("FitnessMax", base.Fitness, weights=(1.0,))
  creator.create("Individual", list, fitness=creator.FitnessMax )
  #initialize and populate toolbox for individuals
  toolbox = base.Toolbox()
  toolbox.register("attr_int", random.randint, 0, 6)
  toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_int, n=IND_SIZE)
  #making population
  toolbox.register("population", tools.initRepeat, list, toolbox.individual)
  pop = toolbox.population(n=POP_SIZE)
  hof = tools.HallOfFame(1)
  toolbox.register("mate", cxSmartGeo)
  toolbox.register("mutate", tools.mutUniformInt, low=0, up=6, indpb=4/100)
  toolbox.register("select", tools.selTournament, tournsize=20)
  toolbox.register("evaluate", evaluate)

  #Registering descriptive statistics
  stats = tools.Statistics(lambda ind: ind.fitness.values)
  stats.register("min", numpy.min)
  stats.register("avg", numpy.mean)
  stats.register("max", numpy.max)
  stats.register("std", numpy.std)

  #Aglorithm for running Simple EA
  algorithms.eaSimple(pop, toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=NGEN, stats=stats, halloffame=hof, verbose=True)
  for i in range(IND_SIZE): 
    stand = tract[i]
   # stand.tcut = best_ind[i]
  for i in range (1):
    best_ind = hof[i]
    print ("best individual", hof[i])
    score = 0
    for i in range (1, TMAX+1):
      score += totalCutPerT(best_ind, i)
      print ("P ", i, ": ", totalCutPerT(best_ind, i))
    print (score)
    print ("evenFlow: ", evenFlow(best_ind))
    print ("adj Violated: ", adjViolated(best_ind))
    print (evenFlowDifference(best_ind))
    for t in range (1, TMAX+1):
      print("In Bounds:", inBounds(best_ind, t))

    best_log ={"Fitness":evaluate(best_ind),
               "Ind": best_ind}
    final+=[best_log]

    

init_Tract("/content/_Lincoln_stand_data.txt","/content/random_adj_list4.txt" )
for i in range(30):
  print("############################# RUN ", i, " #################################")
  createEA()
for i in range(len(final)):
  print(final[i])
  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
############################# RUN  0  #################################
gen	nevals	min        	avg         	max         	std        
0  	200   	-8.6278e+06	-5.72837e+06	-3.24814e+06	1.01473e+06
1  	35    	-4.82954e+06	-3.95012e+06	-2.86144e+06	428387     
2  	36    	-4.29274e+06	-3.2085e+06 	-2.84286e+06	210639     
3  	37    	-3.73762e+06	-2.88276e+06	-2.36585e+06	117744     
4  	43    	-3.50596e+06	-2.7478e+06 	-2.12803e+06	213627     
5  	29    	-3.32229e+06	-2.36484e+06	-1.71426e+06	148327     
6  	51    	-4.05472e+06	-2.16941e+06	-1.64804e+06	295451     
7  	43    	-2.91207e+06	-1.79663e+06	-1.37311e+06	200734     
8  	40    	-2.96321e+06	-1.6419e+06 	-1.29286e+06	252708     
9  	40    	-2.99022e+06	-1.40395e+06	-1.13612e+06	191845     
10 	52    	-2.7675e+06 	-1.33271e+06	-924955     	252245     
11 	42    	-2.22392e+06	-1.14695e+06	-877611     	213124     
12 	33    	-2.2183e+06 	-

In [ ]:
from deap.tools.crossover import cxTwoPoint
"""Lincoln Tract Project"""
!pip install deap

from deap import tools, creator, base, algorithms
import random
import numpy
import math

IND_SIZE = 87
POP_SIZE = 100
TMIN = 0
TMAX = 6
tract = []
CXPB= 0.9
MUTPB = 0.2
NGEN = 100
class Stand:
  def __init__(self, ID, area, age, vacre, tcut, adjmatrix):
    self.ID = ID
    self.area = area
    self.age = age
    self.vacre = vacre
    self.tcut = tcut
    self.adjmatrix = adjmatrix
  def areaCut(self) -> float:
    if self.tcut == 0:
      return 0
    time_cut = self.tcut - 1
    return float(self.area) * float(self.vacre[time_cut])

def feasibleStand(ind, i):
  for n in tract[i].adjmatrix:
    if ind[i] == ind[n-1]:
      return False
  return True

def init_Tract(stand_data, adj_matrix):
  matrix = []
  with open(stand_data) as f:
    lines = f.readlines()
    #assigning stand data to stand classes, populate tract
  for line in lines:
    line = line.rstrip()
    line = line.split(",")
    stand = Stand(line[0], line[1], line[2], [line[3],],\
                  random.randint(TMIN, TMAX), [])
    #assigning volume per acre
    for n in range(4,len(line)):
      vacre = line[n]
      stand.vacre[len(stand.vacre):] = [vacre]
    tract[len(tract):] = [stand]
  #popiulate adjacency matrices
  with open(adj_matrix) as f:
    ajd_lines = f.readlines()
  for line in ajd_lines:
    res = tuple(map(int, line.split(',')))
    matrix[len(matrix):] = [res]
  for stand in tract:
    for pair in matrix:
        if pair[0] == int(stand.ID):
          stand.adjmatrix[len(stand.adjmatrix):] = [pair[1]]

def totalCutPerT(ind, n):
  """Returns total harvest volume of the given time period"""
  if n ==0:
    return 0
  score = 0
  tract_copy = tract.copy()
  for i in range(IND_SIZE):
    stand = tract_copy[i]
    stand.tcut = ind[i]
    if stand.tcut == n:
      score += stand.areaCut()
  return score

def adjViolated(ind):
  """Returns number of adjacency constraints violated"""
  score = 0
  for i in range (IND_SIZE):
    stand = tract[i]
    for n in stand.adjmatrix:
      if stand.tcut > 0 and tract[n-1].tcut == stand.tcut:
        score +=1
  return score

def evaluate(ind):
  """Evaluation function"""
  score=0
  # if adjViolated(ind) == 2 or adjViolated(ind) == 4:
  #   for i in range (IND_SIZE):
  #     stand = tract[i]
  #     for n in stand.adjmatrix:
  #       if tract[n-1] == stand.tcut:
  #         stand.tcut = 0
  #         ind[i] = 0
  for t in range(1, TMAX+1):
    score += totalCutPerT(ind, t)
  #penalties
  if not feasible(ind):
    score = score - 8000 * (adjViolated(ind))
  for t in range (1, TMAX+1):
    if not inBounds(ind, t):
      score = score - 100* sumOutOfBounds(ind, t)
  if not evenFlow(ind):
    score = score- evenFlowDifference(ind)
  return (score,)

def mutInvert(ind):
  """Inverts a sublist"""
  min = random.randint(0, IND_SIZE)
  max = random.randint(0, IND_SIZE)
  if max <= min:
    x = max
    max = min
    min = x
  ind[min:max] = ind[min:max][::-1]
  return (ind,)
    
def evenFlow(ind):
  """Checks to see if there is even flow over the time periods"""
  total_flow = 0
  for num in range(1, TMAX+1):
    total_flow += totalCutPerT(ind, num)
  avg_flow = total_flow/TMAX
  for num in range(1, TMAX+1):
    if totalCutPerT(ind, num) > 1.2 * avg_flow\
    or totalCutPerT(ind, num) < .8 * avg_flow:
      return False
  return True

def evenFlowDifference(ind):
  """Calculates the difference between the avg flow and the actual flow"""
  sum = 0
  total_flow = 0
  for num in range(1, TMAX+1):
    total_flow += totalCutPerT(ind, num)
  avg_flow = total_flow/TMAX
  for num in range(1, TMAX+1):
    if totalCutPerT(ind, num) > 1.2 * avg_flow\
    or totalCutPerT(ind, num) < .8 * avg_flow:
      sum += abs(totalCutPerT(ind, num) - avg_flow)
  return sum


def sumOutOfBounds(ind, t):
  """Returns the difference between the target volume and the
  harvest volume"""
  return abs(13950 - totalCutPerT(ind, t))
  

def inBounds(ind, t):
  """Checks if the amount cut in t is within out maximum harvest volume"""
  volume = 0
  if totalCutPerT(ind, t) > 13950:
    return False
  return True

def feasible(ind):
  """Checks adjacency constraints"""
  for i in range (IND_SIZE):
    stand = tract[i]
    for id in stand.adjmatrix:
      if tract[id-1].tcut == stand.tcut and \
      float(tract[id-1].vacre[tract[id-1].tcut-1]) > 0.0:
          return False
  return True 

def cxSmartAdjCycle(ind1, ind2):

  def addCycle(parent, child, i, count, search_list, removed=[]):
    if count == 0:
      return child
    else:
      parent_stand = tract[i]
      if feasibleStand(parent, i) and child[i] == None:
        child[i] = parent[i]
        # print ("p:",parent)
        # print ("c:", child)
      for stand in parent_stand.adjmatrix:
        if stand not in search_list:
          search_list += [stand]
      for stand in search_list:
        search_list.pop(0)
        return addCycle(parent, child, stand-1, count-1, search_list)

  c1 = ind1.copy()
  c2 = ind2.copy()
  for i in range(IND_SIZE):
    c1[i] = None
    c2[i] = None

  index = random.randint(0, IND_SIZE-1)
  for i in range (index, IND_SIZE):
    if feasibleStand(ind1, i):
      index = i
      break
  #add valid stands
  addCycle(ind1, c1, index, random.randint(0, IND_SIZE), [])
  for i in range(IND_SIZE):
    if c1[i]==None:
      c1[i] = ind2[i]
  for i in range (index, IND_SIZE):
    if feasibleStand(ind2, i):
      index = i
      break

  addCycle(ind2, c2, index, random.randint(0, IND_SIZE), [])
  for i in range(IND_SIZE):
    if c2[i]==None:
      c2[i] = ind1[i]
  # print ("c1:", c1)
  # print ("c2:", c2)
  # print ("i1:", ind1)
  # print ("i2: ", ind2)
  for i in range(IND_SIZE):
    ind1[i] = c1[i]
    ind2[i] = c2[i]
  return (ind1, ind2)

def createEA():
  #Making containers
  creator.create("FitnessMax", base.Fitness, weights=(1.0,))
  creator.create("Individual", list, fitness=creator.FitnessMax )
  #initialize and populate toolbox for individuals
  toolbox = base.Toolbox()
  toolbox.register("attr_int", random.randint, 0, 6)
  toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_int, n=IND_SIZE)
  #making population
  toolbox.register("population", tools.initRepeat, list, toolbox.individual)
  pop = toolbox.population(n=POP_SIZE)
  hof = tools.HallOfFame(1)
  toolbox.register("mate", cxSmartAdjCycle)
  toolbox.register("mutate", mutInvert)
  toolbox.register("mutate", tools.mutUniformInt, low=0, up=6, indpb=4/100)
  toolbox.register("select", tools.selTournament, tournsize=15)
  toolbox.register("evaluate", evaluate)
  #toolbox.decorate("evaluate", tools.DeltaPenalty(feasible, 100, adjViolated))

  #Registering descriptive statistics
  stats = tools.Statistics(lambda ind: ind.fitness.values)
  stats.register("min", numpy.min)
  stats.register("avg", numpy.mean)
  stats.register("max", numpy.max)
  stats.register("std", numpy.std)

  #Aglorithm for running Simple EA
  algorithms.eaSimple(pop, toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=NGEN, stats=stats, halloffame=hof, verbose=True)
  for i in range(IND_SIZE): 
    stand = tract[i]
  for i in range (1):
    best_ind = hof[i]
    print ("best individual", hof[i])
    score = 0
    for i in range (1, TMAX+1):
      score += totalCutPerT(best_ind, i)
      print ("P ", i, ": ", totalCutPerT(best_ind, i))
    print (score)
    print ("evenFlow: ", evenFlow(best_ind))
    print ("adj Violated: ", adjViolated(best_ind))
    print (evenFlowDifference(best_ind))
    for t in range (1, TMAX+1):
      print("In Bounds:", inBounds(best_ind, t))

    

init_Tract("/content/_Lincoln_stand_data.txt","/content/_Lincoln_adjacency_list.txt" )
createEA()


ModuleNotFoundError: ignored

In [ ]:
"""Generate random adjacency list"""
import random
adj_list = []
for _ in range (400):
  x = str(random.randint(0,87))
  y = str(random.randint(0,87))
  while y == x:
    y = str(random.randint(0,87))
  adj_list += [[x, y]]
  adj_list += [[y, x]]
with open('random_adj_list4.txt','w') as f:
  for line in adj_list:
    f.writelines(", ".join(line))
    f.writelines('\n')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')